In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%%capture
!pip install transformers[torch]
!pip install datasets
!pip install pyarrow
!pip install evaluate

In [ ]:
import pandas as pd

In [ ]:
my_data = pd.read_csv("/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/training_data.csv")

In [ ]:
my_data.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


In [ ]:
my_data.shape

(4800, 3)

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import CamembertTokenizer
from typing import Dict

In [ ]:
%%capture
tokenizer = AutoTokenizer.from_pretrained('camembert-base')

In [ ]:
def process_data(row) -> Dict:
    # Clean the text
    text = row['sentence']
    text = str(text)
    text = ' '.join(text.split())

    # Get tokens
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=512)

    # Convert difficulty labels to integers
    difficulty_mapping = {'A1': 0, 'A2': 1, 'B1': 2, 'B2': 3, 'C1': 4, 'C2': 5}
    label = difficulty_mapping.get(row['difficulty'], 0)

    encodings['label'] = label
    encodings['text'] = text

    return encodings

In [ ]:
#test
print(process_data({'sentence': "J'aime les films d'actions",'difficulty': 'B2'}))

{'input_ids': [5, 121, 11, 660, 19, 1379, 18, 11, 6379, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# Store the encodings into an array to generate dataset
processed_data = []

for i in range(len(my_data[:4800])):
        processed_data.append(process_data(my_data.iloc[i]))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(new_df,test_size=0.2,random_state=2022)

In [ ]:
import pyarrow as pa
from datasets import Dataset

In [ ]:
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [ ]:
print(type(train_hg))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
%%capture
model1 = AutoModelForSequenceClassification.from_pretrained('camembert-base',num_labels=6)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=labels)

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/result_V2", evaluation_strategy="epoch")

trainer = Trainer(model=model1,args=training_args,train_dataset=train_hg,eval_dataset=valid_hg,tokenizer=tokenizer,compute_metrics=compute_metrics)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.442195,0.541667,0.549534,0.541667,0.539404
2,0.625600,1.573585,0.576042,0.591141,0.576042,0.576393
3,0.379200,1.926348,0.556250,0.588548,0.556250,0.554875


TrainOutput(global_step=1440, training_loss=0.4209009011586507, metrics={'train_runtime': 1155.3682, 'train_samples_per_second': 9.971, 'train_steps_per_second': 1.246, 'total_flos': 3031148215664640.0, 'train_loss': 0.4209009011586507, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 1.926348328590393,
 'eval_accuracy': 0.55625,
 'eval_precision': 0.5885482243240117,
 'eval_recall': 0.55625,
 'eval_f1': 0.5548746000654246,
 'eval_runtime': 28.6108,
 'eval_samples_per_second': 33.554,
 'eval_steps_per_second': 4.194,
 'epoch': 3.0}

In [ ]:
model1.save_pretrained('/content/drive/MyDrive/Data Science and Machine Learning/Project_Data_Science/train_model_bert_V2/')